# Tutorial Lengkap Sungrow API 🌞

## Panduan Komprehensif untuk Mengakses dan Menggunakan Sungrow iSolarCloud API

---

### Tentang Tutorial Ini

Tutorial ini akan mengajarkan Anda secara lengkap bagaimana menggunakan Sungrow API untuk:
- 📊 Monitoring data real-time dari inverter dan power station
- 📈 Mengakses data historis (produksi energi, performa)
- ⚡ Manajemen dan monitoring perangkat
- 🔔 Mengakses alarm dan event logs
- 📉 Visualisasi data energi
- 💾 Export data untuk analisis lebih lanjut

### Apa itu Sungrow?

**Sungrow Power Supply Co., Ltd** adalah salah satu produsen inverter surya terbesar di dunia. Mereka menyediakan platform monitoring bernama **iSolarCloud** yang memungkinkan pengguna untuk memantau performa sistem solar PV mereka.

### Apa itu Sungrow API?

Sungrow API adalah interface yang memungkinkan developer untuk mengakses data dari platform iSolarCloud secara programmatic. Dengan API ini, Anda bisa:
- Mengintegrasikan data Sungrow ke aplikasi custom
- Membuat dashboard monitoring sendiri
- Menganalisis data energi secara mendalam
- Membuat sistem alerting otomatis

---

**Catatan Penting:** 
- API ini menggunakan REST architecture
- Authentication menggunakan username/password atau App Key
- Rate limiting mungkin diterapkan tergantung akun Anda
- Data yang tersedia tergantung pada perangkat dan konfigurasi sistem Anda

## 📦 Bagian 1: Instalasi dan Setup

### Library yang Dibutuhkan

Untuk bekerja dengan Sungrow API, kita membutuhkan beberapa library Python:

1. **requests** - untuk HTTP requests ke API
2. **pandas** - untuk manipulasi dan analisis data
3. **matplotlib / seaborn** - untuk visualisasi
4. **datetime** - untuk handling waktu (built-in)
5. **json** - untuk parsing response (built-in)

### Catatan Penting

Sungrow tidak menyediakan official Python SDK, jadi kita akan membuat wrapper sendiri menggunakan `requests` library untuk berinteraksi dengan REST API mereka.

In [ ]:
# Install library yang dibutuhkan
# Jalankan cell ini sekali saja saat pertama kali setup

!pip install requests pandas matplotlib seaborn python-dateutil

## 📚 Bagian 2: Import Library

In [ ]:
# Import semua library yang dibutuhkan
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any
import time
import warnings

# Setup untuk visualisasi
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set display options untuk pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✅ Semua library berhasil di-import!")
print(f"📅 Tanggal sekarang: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🔐 Bagian 3: Membuat Sungrow API Client Class

Kita akan membuat sebuah class Python yang akan berfungsi sebagai wrapper untuk Sungrow API. Class ini akan menangani:

1. **Authentication** - Login dan token management
2. **Request handling** - Mengirim request ke API
3. **Error handling** - Menangani error dengan proper
4. **Data parsing** - Mengubah response menjadi format yang mudah digunakan

### Endpoint API Sungrow

Base URL untuk Sungrow iSolarCloud API:
- **Global**: `https://gateway.isolarcloud.com`
- **Eropa**: `https://gateway.isolarcloud.eu`
- **Australia**: `https://augateway.isolarcloud.com`

### Endpoint Penting:
- `/v1/userService/login` - Login
- `/v1/powerStationService/getPsList` - Daftar power station
- `/v1/devService/getDevList` - Daftar device
- `/v1/devService/getCurrentData` - Data real-time
- `/v1/devService/getHisData` - Data historis
- `/v1/alarmService/getAlarmList` - Daftar alarm

In [ ]:
class SungrowAPI:
    """
    Sungrow iSolarCloud API Client
    
    Class ini menyediakan interface untuk berinteraksi dengan Sungrow iSolarCloud API.
    
    Attributes:
        base_url (str): Base URL untuk API endpoint
        username (str): Username untuk login
        password (str): Password untuk login
        token (str): Authentication token setelah login berhasil
        session (requests.Session): Session object untuk maintain connection
    """
    
    def __init__(self, username: str, password: str, region: str = "global"):
        """
        Initialize Sungrow API Client
        
        Args:
            username: Username iSolarCloud
            password: Password iSolarCloud
            region: Region server ('global', 'eu', 'au'). Default: 'global'
        """
        # Set base URL berdasarkan region
        regions = {
            "global": "https://gateway.isolarcloud.com",
            "eu": "https://gateway.isolarcloud.eu",
            "au": "https://augateway.isolarcloud.com"
        }
        
        self.base_url = regions.get(region, regions["global"])
        self.username = username
        self.password = password
        self.token = None
        self.user_id = None
        self.session = requests.Session()
        
        # Set default headers
        self.session.headers.update({
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        })
        
        print(f"✅ Sungrow API Client initialized dengan region: {region}")
        print(f"🌐 Base URL: {self.base_url}")
    
    def login(self) -> bool:
        """
        Login ke Sungrow API dan dapatkan token
        
        Returns:
            bool: True jika login berhasil, False jika gagal
        """
        endpoint = "/v1/userService/login"
        url = f"{self.base_url}{endpoint}"
        
        payload = {
            "user_account": self.username,
            "user_password": self.password
        }
        
        try:
            print("🔐 Mencoba login...")
            response = self.session.post(url, json=payload, timeout=30)
            response.raise_for_status()
            
            data = response.json()
            
            if data.get('result_code') == 1:
                self.token = data['result_data']['token']
                self.user_id = data['result_data'].get('user_id')
                
                # Update headers dengan token
                self.session.headers.update({
                    'token': self.token
                })
                
                print("✅ Login berhasil!")
                print(f"👤 User ID: {self.user_id}")
                return True
            else:
                print(f"❌ Login gagal: {data.get('result_msg', 'Unknown error')}")
                return False
                
        except requests.exceptions.RequestException as e:
            print(f"❌ Error saat login: {str(e)}")
            return False
    
    def _make_request(self, endpoint: str, method: str = "POST", 
                     params: Optional[Dict] = None) -> Optional[Dict]:
        """
        Helper method untuk membuat request ke API
        
        Args:
            endpoint: API endpoint
            method: HTTP method (GET/POST)
            params: Request parameters
            
        Returns:
            Dict: Response data atau None jika error
        """
        if not self.token:
            print("❌ Belum login! Silakan login terlebih dahulu.")
            return None
        
        url = f"{self.base_url}{endpoint}"
        
        try:
            if method.upper() == "POST":
                response = self.session.post(url, json=params, timeout=30)
            else:
                response = self.session.get(url, params=params, timeout=30)
            
            response.raise_for_status()
            data = response.json()
            
            if data.get('result_code') == 1:
                return data.get('result_data')
            else:
                print(f"❌ API Error: {data.get('result_msg', 'Unknown error')}")
                return None
                
        except requests.exceptions.RequestException as e:
            print(f"❌ Request error: {str(e)}")
            return None
    
    def get_power_stations(self) -> Optional[List[Dict]]:
        """
        Dapatkan daftar power station yang tersedia
        
        Returns:
            List[Dict]: List of power stations
        """
        endpoint = "/v1/powerStationService/getPsList"
        data = self._make_request(endpoint)
        
        if data and 'pageList' in data:
            return data['pageList']
        return None
    
    def get_device_list(self, ps_id: str) -> Optional[List[Dict]]:
        """
        Dapatkan daftar device dalam suatu power station
        
        Args:
            ps_id: Power Station ID
            
        Returns:
            List[Dict]: List of devices
        """
        endpoint = "/v1/devService/getDevList"
        params = {"ps_id": ps_id}
        data = self._make_request(endpoint, params=params)
        
        if data and 'list' in data:
            return data['list']
        return None
    
    def get_realtime_data(self, dev_id: str, dev_type: int = 1) -> Optional[Dict]:
        """
        Dapatkan data real-time dari device
        
        Args:
            dev_id: Device ID
            dev_type: Device type (1=Inverter, 2=Logger, etc)
            
        Returns:
            Dict: Realtime data
        """
        endpoint = "/v1/devService/getCurrentData"
        params = {
            "dev_id": dev_id,
            "dev_type": dev_type
        }
        return self._make_request(endpoint, params=params)
    
    def get_historical_data(self, dev_id: str, dev_type: int, 
                          date_start: str, date_end: str, 
                          date_type: int = 2) -> Optional[Dict]:
        """
        Dapatkan data historis
        
        Args:
            dev_id: Device ID
            dev_type: Device type
            date_start: Start date (format: yyyy-MM-dd)
            date_end: End date (format: yyyy-MM-dd)
            date_type: 1=Minute, 2=Hour, 3=Day, 4=Month
            
        Returns:
            Dict: Historical data
        """
        endpoint = "/v1/devService/getHisData"
        params = {
            "dev_id": dev_id,
            "dev_type": dev_type,
            "start_time": date_start,
            "end_time": date_end,
            "date_type": date_type
        }
        return self._make_request(endpoint, params=params)
    
    def get_ps_detail(self, ps_id: str) -> Optional[Dict]:
        """
        Dapatkan detail power station
        
        Args:
            ps_id: Power Station ID
            
        Returns:
            Dict: Power station details
        """
        endpoint = "/v1/powerStationService/getPsDetail"
        params = {"ps_id": ps_id}
        return self._make_request(endpoint, params=params)
    
    def get_alarm_list(self, ps_id: str, language: int = 2) -> Optional[List[Dict]]:
        """
        Dapatkan daftar alarm
        
        Args:
            ps_id: Power Station ID
            language: 1=Chinese, 2=English
            
        Returns:
            List[Dict]: List of alarms
        """
        endpoint = "/v1/alarmService/getAlarmList"
        params = {
            "ps_id": ps_id,
            "language": language
        }
        data = self._make_request(endpoint, params=params)
        
        if data and 'pageList' in data:
            return data['pageList']
        return None

print("✅ Class SungrowAPI berhasil dibuat!")

## 🔑 Bagian 4: Konfigurasi dan Authentication

Sekarang kita akan setup credentials dan melakukan login ke Sungrow API.

### ⚠️ PENTING - Keamanan Credentials

**JANGAN PERNAH** hardcode username dan password di production code! Gunakan salah satu metode berikut:

1. **Environment Variables** (Recommended)
2. **Config file** yang di-ignore di git (.env file)
3. **Secret management service** (AWS Secrets Manager, Azure Key Vault, etc)

Untuk tutorial ini, kita akan menggunakan variabel langsung, tapi **pastikan Anda tidak commit credentials ke Git!**

In [ ]:
# ========================================
# KONFIGURASI CREDENTIALS
# ========================================
# GANTI dengan credentials Anda sendiri!

USERNAME = "your_username@email.com"  # Ganti dengan username Anda
PASSWORD = "your_password"             # Ganti dengan password Anda
REGION = "global"                      # Options: "global", "eu", "au"

# Alternatif: Menggunakan environment variables (lebih aman)
# import os
# USERNAME = os.getenv('SUNGROW_USERNAME', 'default_username')
# PASSWORD = os.getenv('SUNGROW_PASSWORD', 'default_password')

print("⚙️ Konfigurasi:")
print(f"   Username: {USERNAME}")
print(f"   Password: {'*' * len(PASSWORD)}")  # Tidak menampilkan password asli
print(f"   Region: {REGION}")

In [ ]:
# ========================================
# INISIALISASI CLIENT DAN LOGIN
# ========================================

# Buat instance SungrowAPI client
client = SungrowAPI(
    username=USERNAME,
    password=PASSWORD,
    region=REGION
)

# Login ke API
login_success = client.login()

if login_success:
    print("\n🎉 Berhasil terhubung ke Sungrow API!")
    print("    Anda sekarang bisa mulai menggunakan API.")
else:
    print("\n❌ Gagal login ke Sungrow API!")
    print("    Pastikan username dan password Anda benar.")

## 🏭 Bagian 5: Mendapatkan Daftar Power Station

Power Station adalah lokasi fisik dimana sistem solar PV Anda terpasang. Satu akun bisa memiliki multiple power stations.

Mari kita dapatkan daftar power station yang terhubung dengan akun Anda.

In [ ]:
# Dapatkan daftar power station
power_stations = client.get_power_stations()

if power_stations:
    print(f"📊 Total Power Station: {len(power_stations)}\n")
    
    # Simpan dalam DataFrame untuk tampilan yang lebih baik
    df_ps = pd.DataFrame(power_stations)
    
    # Tampilkan informasi setiap power station
    for idx, ps in enumerate(power_stations, 1):
        print(f"{'='*60}")
        print(f"Power Station #{idx}")
        print(f"{'='*60}")
        print(f"  🏭 Name: {ps.get('ps_name', 'N/A')}")
        print(f"  🆔 ID: {ps.get('ps_id', 'N/A')}")
        print(f"  ⚡ Total Capacity: {ps.get('ps_capacity', 'N/A')} kW")
        print(f"  📍 Location: {ps.get('ps_location', 'N/A')}")
        print(f"  📈 Total Power Generated: {ps.get('total_power', 'N/A')} kWh")
        print(f"  💰 Total Revenue: {ps.get('total_revenue', 'N/A')}")
        print(f"  🌱 CO2 Reduction: {ps.get('co2_reduction', 'N/A')} tons")
        print()
    
    # Tampilkan DataFrame
    print("\n📋 Tabel Power Station:")
    display(df_ps)
    
    # Simpan PS_ID pertama untuk digunakan di cell selanjutnya
    if len(power_stations) > 0:
        SELECTED_PS_ID = power_stations[0]['ps_id']
        print(f"\n✅ Selected PS_ID untuk contoh selanjutnya: {SELECTED_PS_ID}")
else:
    print("❌ Tidak ada power station ditemukan atau gagal mengambil data.")

## 📱 Bagian 6: Mendapatkan Daftar Device

Setiap power station memiliki satu atau lebih device (biasanya inverter). Mari kita dapatkan daftar device.

In [ ]:
# Dapatkan daftar device dari power station yang dipilih
if 'SELECTED_PS_ID' in locals():
    devices = client.get_device_list(SELECTED_PS_ID)
    
    if devices:
        print(f"📱 Total Devices: {len(devices)}\n")
        
        # Simpan dalam DataFrame
        df_devices = pd.DataFrame(devices)
        
        # Tampilkan informasi setiap device
        for idx, dev in enumerate(devices, 1):
            print(f"{'='*60}")
            print(f"Device #{idx}")
            print(f"{'='*60}")
            print(f"  📱 Device Name: {dev.get('dev_name', 'N/A')}")
            print(f"  🆔 Device ID: {dev.get('dev_id', 'N/A')}")
            print(f"  🔧 Device Type: {dev.get('dev_type', 'N/A')}")
            print(f"  📊 Model: {dev.get('dev_model', 'N/A')}")
            print(f"  🔢 Serial Number: {dev.get('dev_sn', 'N/A')}")
            print(f"  📡 Status: {dev.get('dev_status', 'N/A')}")
            print()
        
        # Tampilkan DataFrame
        print("\n📋 Tabel Devices:")
        display(df_devices)
        
        # Simpan device pertama untuk contoh selanjutnya
        if len(devices) > 0:
            SELECTED_DEV_ID = devices[0]['dev_id']
            SELECTED_DEV_TYPE = devices[0].get('dev_type', 1)
            print(f"\n✅ Selected Device ID: {SELECTED_DEV_ID}")
            print(f"✅ Selected Device Type: {SELECTED_DEV_TYPE}")
    else:
        print("❌ Tidak ada device ditemukan atau gagal mengambil data.")
else:
    print("⚠️ PS_ID belum tersedia. Jalankan cell sebelumnya terlebih dahulu.")

## ⚡ Bagian 7: Mendapatkan Data Real-time

Data real-time memberikan informasi tentang kondisi device saat ini, termasuk:
- Power output (daya yang dihasilkan saat ini)
- Voltage (tegangan)
- Current (arus)
- Temperature (suhu)
- Status operasional
- Dan parameter lainnya

Mari kita ambil data real-time dari device yang dipilih.

In [ ]:
# Dapatkan data real-time
if 'SELECTED_DEV_ID' in locals():
    realtime_data = client.get_realtime_data(
        dev_id=SELECTED_DEV_ID,
        dev_type=SELECTED_DEV_TYPE
    )
    
    if realtime_data:
        print(f"⚡ DATA REAL-TIME")
        print(f"{'='*60}\n")
        
        # Data biasanya dalam format dictionary atau list
        if isinstance(realtime_data, dict):
            # Jika data berupa dict dengan key 'list' atau similar
            data_list = realtime_data.get('list', [realtime_data])
        elif isinstance(realtime_data, list):
            data_list = realtime_data
        else:
            data_list = [realtime_data]
        
        # Tampilkan setiap parameter
        for item in data_list:
            if isinstance(item, dict):
                param_name = item.get('param_name', item.get('name', 'N/A'))
                param_value = item.get('param_value', item.get('value', 'N/A'))
                param_unit = item.get('param_unit', item.get('unit', ''))
                
                print(f"  📊 {param_name}: {param_value} {param_unit}")
        
        print(f"\n{'='*60}")
        
        # Konversi ke DataFrame untuk analisis lebih lanjut
        try:
            df_realtime = pd.DataFrame(data_list)
            print("\n📋 Data dalam bentuk tabel:")
            display(df_realtime)
            
            # Simpan untuk digunakan nanti
            REALTIME_DF = df_realtime
        except Exception as e:
            print(f"\n⚠️ Tidak bisa konversi ke DataFrame: {e}")
            print("\n📝 Raw data:")
            print(json.dumps(realtime_data, indent=2, ensure_ascii=False))
    else:
        print("❌ Gagal mengambil data real-time.")
else:
    print("⚠️ Device ID belum tersedia. Jalankan cell sebelumnya terlebih dahulu.")

## 📈 Bagian 8: Mendapatkan Data Historis

Data historis memungkinkan kita untuk melihat performa sistem dalam periode waktu tertentu. Kita bisa mengambil data dengan berbagai interval:

- **date_type = 1**: Data per menit
- **date_type = 2**: Data per jam (hourly)
- **date_type = 3**: Data per hari (daily)
- **date_type = 4**: Data per bulan (monthly)

Mari kita ambil data historis untuk 7 hari terakhir dengan interval per jam.

In [ ]:
# Setup tanggal untuk historical data
# Contoh: ambil data 7 hari terakhir
end_date = datetime.now()
start_date = end_date - timedelta(days=7)

# Format tanggal ke format yang dibutuhkan API (yyyy-MM-dd)
date_start = start_date.strftime('%Y-%m-%d')
date_end = end_date.strftime('%Y-%m-%d')

print(f"📅 Periode Data:")
print(f"   Start: {date_start}")
print(f"   End: {date_end}")
print(f"   Duration: 7 days")

# Ambil data historis
if 'SELECTED_DEV_ID' in locals():
    historical_data = client.get_historical_data(
        dev_id=SELECTED_DEV_ID,
        dev_type=SELECTED_DEV_TYPE,
        date_start=date_start,
        date_end=date_end,
        date_type=2  # 2 = hourly data
    )
    
    if historical_data:
        print("\n✅ Data historis berhasil diambil!")
        
        # Parse data
        if isinstance(historical_data, dict):
            data_list = historical_data.get('list', [])
            
            if data_list:
                print(f"📊 Total data points: {len(data_list)}")
                
                # Konversi ke DataFrame
                df_historical = pd.DataFrame(data_list)
                
                # Tampilkan preview data
                print("\n📋 Preview Data (5 baris pertama):")
                display(df_historical.head())
                
                print("\n📋 Preview Data (5 baris terakhir):")
                display(df_historical.tail())
                
                # Info tentang DataFrame
                print("\n📊 Info DataFrame:")
                print(df_historical.info())
                
                print("\n📈 Statistik Deskriptif:")
                display(df_historical.describe())
                
                # Simpan untuk analisis lebih lanjut
                HISTORICAL_DF = df_historical
            else:
                print("\n⚠️ Data list kosong")
        else:
            print("\n📝 Raw data:")
            print(json.dumps(historical_data, indent=2, ensure_ascii=False))
    else:
        print("\n❌ Gagal mengambil data historis.")
else:
    print("⚠️ Device ID belum tersedia. Jalankan cell sebelumnya terlebih dahulu.")

## 🏭 Bagian 9: Detail Power Station

Mari kita dapatkan informasi detail tentang power station, termasuk statistik kumulatif dan informasi teknis.

In [ ]:
# Dapatkan detail power station
if 'SELECTED_PS_ID' in locals():
    ps_detail = client.get_ps_detail(SELECTED_PS_ID)
    
    if ps_detail:
        print(f"🏭 DETAIL POWER STATION")
        print(f"{'='*70}\n")
        
        # Tampilkan informasi umum
        print(f"📋 Informasi Umum:")
        print(f"{'─'*70}")
        print(f"  Name: {ps_detail.get('ps_name', 'N/A')}")
        print(f"  ID: {ps_detail.get('ps_id', 'N/A')}")
        print(f"  Location: {ps_detail.get('ps_location', 'N/A')}")
        print(f"  Capacity: {ps_detail.get('ps_capacity', 'N/A')} kW")
        print(f"  Type: {ps_detail.get('ps_type', 'N/A')}")
        print()
        
        # Statistik produksi
        print(f"⚡ Statistik Produksi:")
        print(f"{'─'*70}")
        print(f"  Current Power: {ps_detail.get('current_power', 'N/A')} kW")
        print(f"  Today Energy: {ps_detail.get('today_energy', 'N/A')} kWh")
        print(f"  Month Energy: {ps_detail.get('month_energy', 'N/A')} kWh")
        print(f"  Year Energy: {ps_detail.get('year_energy', 'N/A')} kWh")
        print(f"  Total Energy: {ps_detail.get('total_energy', 'N/A')} kWh")
        print()
        
        # Informasi finansial
        print(f"💰 Informasi Finansial:")
        print(f"{'─'*70}")
        print(f"  Today Revenue: {ps_detail.get('today_revenue', 'N/A')}")
        print(f"  Total Revenue: {ps_detail.get('total_revenue', 'N/A')}")
        print()
        
        # Informasi lingkungan
        print(f"🌱 Dampak Lingkungan:")
        print(f"{'─'*70}")
        print(f"  CO2 Reduction: {ps_detail.get('co2_reduction', 'N/A')} tons")
        print(f"  Coal Saved: {ps_detail.get('coal_saved', 'N/A')} tons")
        print(f"  Trees Planted Equivalent: {ps_detail.get('tree_planted', 'N/A')}")
        print()
        
        print(f"{'='*70}")
        
        # Tampilkan raw data untuk referensi
        print("\n📝 Raw JSON Data:")
        print(json.dumps(ps_detail, indent=2, ensure_ascii=False))
    else:
        print("❌ Gagal mengambil detail power station.")
else:
    print("⚠️ PS_ID belum tersedia. Jalankan cell sebelumnya terlebih dahulu.")

## 🔔 Bagian 10: Mendapatkan Daftar Alarm dan Event

Alarm dan event logs membantu kita memantau masalah atau kejadian penting pada sistem. Mari kita lihat alarm yang ada.

In [ ]:
# Dapatkan daftar alarm
if 'SELECTED_PS_ID' in locals():
    alarms = client.get_alarm_list(
        ps_id=SELECTED_PS_ID,
        language=2  # 2 = English, 1 = Chinese
    )
    
    if alarms:
        print(f"🔔 DAFTAR ALARM")
        print(f"{'='*70}\n")
        print(f"Total Alarms: {len(alarms)}\n")
        
        if len(alarms) > 0:
            # Konversi ke DataFrame
            df_alarms = pd.DataFrame(alarms)
            
            # Tampilkan setiap alarm
            for idx, alarm in enumerate(alarms[:10], 1):  # Tampilkan max 10 alarm pertama
                print(f"{'─'*70}")
                print(f"Alarm #{idx}")
                print(f"{'─'*70}")
                print(f"  🆔 Alarm ID: {alarm.get('alarm_id', 'N/A')}")
                print(f"  📱 Device: {alarm.get('dev_name', 'N/A')}")
                print(f"  ⚠️ Type: {alarm.get('alarm_type', 'N/A')}")
                print(f"  📝 Message: {alarm.get('alarm_msg', 'N/A')}")
                print(f"  ⚡ Level: {alarm.get('alarm_level', 'N/A')}")
                print(f"  📅 Start Time: {alarm.get('start_time', 'N/A')}")
                print(f"  ✅ End Time: {alarm.get('end_time', 'N/A')}")
                print(f"  📊 Status: {alarm.get('status', 'N/A')}")
                print()
            
            if len(alarms) > 10:
                print(f"\n... dan {len(alarms) - 10} alarm lainnya")
            
            # Tampilkan DataFrame
            print("\n📋 Tabel Alarm:")
            display(df_alarms)
            
            # Analisis alarm
            print("\n📊 Analisis Alarm:")
            print(f"{'─'*70}")
            
            if 'alarm_level' in df_alarms.columns:
                print("\nDistribusi berdasarkan Level:")
                print(df_alarms['alarm_level'].value_counts())
            
            if 'alarm_type' in df_alarms.columns:
                print("\nDistribusi berdasarkan Type:")
                print(df_alarms['alarm_type'].value_counts())
        else:
            print("✅ Tidak ada alarm. Sistem berjalan normal!")
    else:
        print("❌ Gagal mengambil daftar alarm atau tidak ada alarm.")
else:
    print("⚠️ PS_ID belum tersedia. Jalankan cell sebelumnya terlebih dahulu.")

## 📊 Bagian 11: Visualisasi Data

Sekarang mari kita visualisasikan data yang telah kita ambil. Visualisasi akan membantu kita memahami pola dan trend data dengan lebih baik.

### 11.1 Visualisasi Data Historis - Power Generation

In [ ]:
# Visualisasi Data Historis
if 'HISTORICAL_DF' in locals() and not HISTORICAL_DF.empty:
    # Setup figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('📊 Analisis Data Energi - 7 Hari Terakhir', fontsize=16, fontweight='bold')
    
    # Cek kolom yang tersedia
    print("📋 Kolom yang tersedia dalam data:")
    print(HISTORICAL_DF.columns.tolist())
    
    # Coba identifikasi kolom power dan energy
    power_col = None
    energy_col = None
    time_col = None
    
    # Cari kolom yang mungkin berisi data power atau energy
    for col in HISTORICAL_DF.columns:
        col_lower = col.lower()
        if 'power' in col_lower and power_col is None:
            power_col = col
        if 'energy' in col_lower or 'kwh' in col_lower and energy_col is None:
            energy_col = col
        if 'time' in col_lower or 'date' in col_lower and time_col is None:
            time_col = col
    
    # Jika ada kolom time, konversi ke datetime
    if time_col and time_col in HISTORICAL_DF.columns:
        try:
            HISTORICAL_DF[time_col] = pd.to_datetime(HISTORICAL_DF[time_col])
            HISTORICAL_DF = HISTORICAL_DF.sort_values(time_col)
        except:
            pass
    
    # Plot 1: Power Generation Over Time
    if power_col:
        axes[0, 0].plot(HISTORICAL_DF.index if time_col is None else HISTORICAL_DF[time_col], 
                       HISTORICAL_DF[power_col], 
                       color='#FF6B6B', linewidth=2, marker='o', markersize=3)
        axes[0, 0].set_title('⚡ Power Generation Over Time', fontsize=12, fontweight='bold')
        axes[0, 0].set_xlabel('Time')
        axes[0, 0].set_ylabel('Power (kW)')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].tick_params(axis='x', rotation=45)
    else:
        axes[0, 0].text(0.5, 0.5, 'No Power Data Available', 
                       ha='center', va='center', fontsize=12)
    
    # Plot 2: Energy Production (if available)
    if energy_col:
        axes[0, 1].bar(HISTORICAL_DF.index if time_col is None else range(len(HISTORICAL_DF)), 
                      HISTORICAL_DF[energy_col], 
                      color='#4ECDC4', alpha=0.7)
        axes[0, 1].set_title('📈 Energy Production', fontsize=12, fontweight='bold')
        axes[0, 1].set_xlabel('Time Index')
        axes[0, 1].set_ylabel('Energy (kWh)')
        axes[0, 1].grid(True, alpha=0.3, axis='y')
    else:
        axes[0, 1].text(0.5, 0.5, 'No Energy Data Available', 
                       ha='center', va='center', fontsize=12)
    
    # Plot 3: Distribution (histogram)
    if power_col:
        axes[1, 0].hist(HISTORICAL_DF[power_col].dropna(), bins=30, 
                       color='#95E1D3', edgecolor='black', alpha=0.7)
        axes[1, 0].set_title('📊 Power Distribution', fontsize=12, fontweight='bold')
        axes[1, 0].set_xlabel('Power (kW)')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].grid(True, alpha=0.3, axis='y')
    else:
        axes[1, 0].text(0.5, 0.5, 'No Power Data Available', 
                       ha='center', va='center', fontsize=12)
    
    # Plot 4: Summary Statistics
    axes[1, 1].axis('off')
    stats_text = "📊 STATISTIK RINGKASAN\n" + "="*40 + "\n\n"
    
    if power_col and power_col in HISTORICAL_DF.columns:
        power_data = HISTORICAL_DF[power_col].dropna()
        stats_text += f"⚡ Power:\n"
        stats_text += f"   Max: {power_data.max():.2f} kW\n"
        stats_text += f"   Min: {power_data.min():.2f} kW\n"
        stats_text += f"   Mean: {power_data.mean():.2f} kW\n"
        stats_text += f"   Std: {power_data.std():.2f} kW\n\n"
    
    if energy_col and energy_col in HISTORICAL_DF.columns:
        energy_data = HISTORICAL_DF[energy_col].dropna()
        stats_text += f"📈 Energy:\n"
        stats_text += f"   Total: {energy_data.sum():.2f} kWh\n"
        stats_text += f"   Max: {energy_data.max():.2f} kWh\n"
        stats_text += f"   Mean: {energy_data.mean():.2f} kWh\n\n"
    
    stats_text += f"📅 Period:\n"
    stats_text += f"   Total Data Points: {len(HISTORICAL_DF)}\n"
    stats_text += f"   Duration: 7 days\n"
    
    axes[1, 1].text(0.1, 0.9, stats_text, 
                   fontsize=11, verticalalignment='top', 
                   fontfamily='monospace',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Visualisasi berhasil dibuat!")
else:
    print("⚠️ Data historis belum tersedia atau kosong.")
    print("   Jalankan cell pengambilan data historis terlebih dahulu.")

### 11.2 Visualisasi Perbandingan Multiple Power Stations

Jika Anda memiliki multiple power stations, kita bisa membandingkan performa mereka.

In [ ]:
# Visualisasi Multiple Power Stations
if 'power_stations' in locals() and len(power_stations) > 0:
    # Extract data untuk visualisasi
    ps_names = []
    ps_capacity = []
    ps_today_energy = []
    ps_total_energy = []
    
    for ps in power_stations:
        ps_names.append(ps.get('ps_name', 'Unknown')[:20])  # Limit nama untuk display
        
        # Capacity
        capacity = ps.get('ps_capacity', 0)
        try:
            ps_capacity.append(float(capacity) if capacity else 0)
        except:
            ps_capacity.append(0)
        
        # Today energy
        today = ps.get('today_energy', 0)
        try:
            ps_today_energy.append(float(today) if today else 0)
        except:
            ps_today_energy.append(0)
        
        # Total energy
        total = ps.get('total_energy', 0)
        try:
            ps_total_energy.append(float(total) if total else 0)
        except:
            ps_total_energy.append(0)
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('📊 Perbandingan Power Station', fontsize=16, fontweight='bold')
    
    # Plot 1: Capacity Comparison
    axes[0, 0].barh(ps_names, ps_capacity, color='#FF6B6B', alpha=0.7)
    axes[0, 0].set_title('⚡ Kapasitas Terpasang', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Capacity (kW)')
    axes[0, 0].grid(True, alpha=0.3, axis='x')
    
    # Plot 2: Today's Energy
    axes[0, 1].bar(range(len(ps_names)), ps_today_energy, 
                   color='#4ECDC4', alpha=0.7, edgecolor='black')
    axes[0, 1].set_title('📅 Produksi Energi Hari Ini', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Energy (kWh)')
    axes[0, 1].set_xticks(range(len(ps_names)))
    axes[0, 1].set_xticklabels(ps_names, rotation=45, ha='right')
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # Plot 3: Total Energy Pie Chart
    if sum(ps_total_energy) > 0:
        colors = plt.cm.Set3(range(len(ps_names)))
        axes[1, 0].pie(ps_total_energy, labels=ps_names, autopct='%1.1f%%',
                      colors=colors, startangle=90)
        axes[1, 0].set_title('🔄 Proporsi Total Energi', fontsize=12, fontweight='bold')
    else:
        axes[1, 0].text(0.5, 0.5, 'No Total Energy Data', 
                       ha='center', va='center', fontsize=12)
        axes[1, 0].axis('off')
    
    # Plot 4: Summary Table
    axes[1, 1].axis('off')
    summary_data = []
    for i, name in enumerate(ps_names):
        summary_data.append([
            name,
            f"{ps_capacity[i]:.1f}",
            f"{ps_today_energy[i]:.1f}",
            f"{ps_total_energy[i]:.1f}"
        ])
    
    table = axes[1, 1].table(cellText=summary_data,
                            colLabels=['Station', 'Cap(kW)', 'Today(kWh)', 'Total(kWh)'],
                            cellLoc='center',
                            loc='center',
                            bbox=[0, 0, 1, 1])
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2)
    
    # Style header
    for i in range(4):
        table[(0, i)].set_facecolor('#40466e')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    # Alternate row colors
    for i in range(1, len(summary_data) + 1):
        for j in range(4):
            if i % 2 == 0:
                table[(i, j)].set_facecolor('#f0f0f0')
    
    axes[1, 1].set_title('📋 Ringkasan Data', fontsize=12, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualisasi perbandingan power station berhasil dibuat!")
else:
    print("⚠️ Data power station belum tersedia.")

## 💾 Bagian 12: Export Data ke File

Setelah kita mengambil data dari API, seringkali kita ingin menyimpannya untuk analisis lebih lanjut atau reporting. Mari kita export data ke berbagai format.

In [ ]:
import os

# Buat folder untuk menyimpan exports
export_folder = "sungrow_exports"
if not os.path.exists(export_folder):
    os.makedirs(export_folder)
    print(f"✅ Folder '{export_folder}' berhasil dibuat")

# Generate timestamp untuk nama file
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# =========================================
# 1. Export Power Stations ke CSV
# =========================================
if 'df_ps' in locals():
    filename = f"{export_folder}/power_stations_{timestamp}.csv"
    df_ps.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ Power stations data exported: {filename}")

# =========================================
# 2. Export Devices ke CSV
# =========================================
if 'df_devices' in locals():
    filename = f"{export_folder}/devices_{timestamp}.csv"
    df_devices.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ Devices data exported: {filename}")

# =========================================
# 3. Export Historical Data ke CSV
# =========================================
if 'HISTORICAL_DF' in locals():
    filename = f"{export_folder}/historical_data_{timestamp}.csv"
    HISTORICAL_DF.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ Historical data exported: {filename}")

# =========================================
# 4. Export Alarms ke CSV
# =========================================
if 'df_alarms' in locals():
    filename = f"{export_folder}/alarms_{timestamp}.csv"
    df_alarms.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ Alarms data exported: {filename}")

# =========================================
# 5. Export ke Excel (Multiple Sheets)
# =========================================
try:
    excel_filename = f"{export_folder}/sungrow_complete_data_{timestamp}.xlsx"
    
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        if 'df_ps' in locals():
            df_ps.to_excel(writer, sheet_name='Power Stations', index=False)
        
        if 'df_devices' in locals():
            df_devices.to_excel(writer, sheet_name='Devices', index=False)
        
        if 'HISTORICAL_DF' in locals():
            HISTORICAL_DF.to_excel(writer, sheet_name='Historical Data', index=False)
        
        if 'df_alarms' in locals():
            df_alarms.to_excel(writer, sheet_name='Alarms', index=False)
    
    print(f"✅ Excel file exported: {excel_filename}")
except Exception as e:
    print(f"⚠️ Excel export failed: {e}")
    print("   Install openpyxl: pip install openpyxl")

# =========================================
# 6. Export Summary Report ke JSON
# =========================================
summary_report = {
    "export_timestamp": timestamp,
    "export_date": datetime.now().isoformat(),
    "data_summary": {
        "power_stations_count": len(power_stations) if 'power_stations' in locals() else 0,
        "devices_count": len(devices) if 'devices' in locals() else 0,
        "historical_records": len(HISTORICAL_DF) if 'HISTORICAL_DF' in locals() else 0,
        "alarms_count": len(alarms) if 'alarms' in locals() else 0
    }
}

json_filename = f"{export_folder}/export_summary_{timestamp}.json"
with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(summary_report, f, indent=2, ensure_ascii=False)

print(f"✅ Summary report exported: {json_filename}")

print("\n" + "="*60)
print("📁 Semua file berhasil di-export ke folder:", export_folder)
print("="*60)

## 🔄 Bagian 13: Monitoring Otomatis dan Scheduled Tasks

Untuk monitoring real-time dan pengumpulan data berkala, kita bisa membuat fungsi yang berjalan secara otomatis.

### 13.1 Fungsi Helper untuk Monitoring

In [ ]:
def monitor_realtime(client, ps_id, dev_id, dev_type, interval=60, duration=300):
    """
    Monitor real-time data dengan interval tertentu
    
    Args:
        client: SungrowAPI client instance
        ps_id: Power Station ID
        dev_id: Device ID
        dev_type: Device Type
        interval: Interval in seconds between checks
        duration: Total duration in seconds
    """
    print(f"🔄 Starting monitoring...")
    print(f"   Interval: {interval} seconds")
    print(f"   Duration: {duration} seconds ({duration//60} minutes)")
    print(f"   Press Ctrl+C to stop\n")
    
    start_time = time.time()
    data_collected = []
    
    try:
        while (time.time() - start_time) < duration:
            # Get current time
            current_time = datetime.now()
            
            # Get realtime data
            data = client.get_realtime_data(dev_id, dev_type)
            
            if data:
                # Extract key metrics (adjust based on actual data structure)
                record = {
                    'timestamp': current_time.isoformat(),
                    'raw_data': data
                }
                data_collected.append(record)
                
                # Display current status
                print(f"[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] ✅ Data collected")
                
                # Check for alarms
                alarms = client.get_alarm_list(ps_id)
                if alarms and len(alarms) > 0:
                    print(f"   ⚠️ Active alarms: {len(alarms)}")
            else:
                print(f"[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] ❌ Failed to get data")
            
            # Wait for next interval
            time.sleep(interval)
            
    except KeyboardInterrupt:
        print("\n\n⏹️ Monitoring stopped by user")
    
    # Return collected data
    print(f"\n📊 Total data points collected: {len(data_collected)}")
    return data_collected


def check_system_health(client, ps_id):
    """
    Check overall system health
    
    Args:
        client: SungrowAPI client instance
        ps_id: Power Station ID
        
    Returns:
        dict: Health status report
    """
    print("🏥 Checking system health...\n")
    
    health_report = {
        'timestamp': datetime.now().isoformat(),
        'status': 'HEALTHY',
        'issues': []
    }
    
    # Check for active alarms
    alarms = client.get_alarm_list(ps_id)
    if alarms:
        active_alarms = [a for a in alarms if a.get('status') != 'resolved']
        if active_alarms:
            health_report['status'] = 'WARNING'
            health_report['issues'].append(f"Active alarms: {len(active_alarms)}")
            print(f"⚠️ Active alarms detected: {len(active_alarms)}")
    
    # Get PS details
    ps_detail = client.get_ps_detail(ps_id)
    if ps_detail:
        current_power = ps_detail.get('current_power', 0)
        capacity = ps_detail.get('ps_capacity', 1)
        
        try:
            # Check if system is underperforming (less than 5% of capacity during day)
            hour = datetime.now().hour
            if 9 <= hour <= 16:  # During typical sunlight hours
                if float(current_power) < float(capacity) * 0.05:
                    health_report['status'] = 'WARNING'
                    health_report['issues'].append("Low power output during daylight")
                    print(f"⚠️ Low power output: {current_power} kW (Capacity: {capacity} kW)")
        except:
            pass
    
    # Get device list and check status
    devices = client.get_device_list(ps_id)
    if devices:
        offline_devices = [d for d in devices if d.get('dev_status') == 'offline']
        if offline_devices:
            health_report['status'] = 'CRITICAL'
            health_report['issues'].append(f"Offline devices: {len(offline_devices)}")
            print(f"❌ Offline devices detected: {len(offline_devices)}")
    
    # Summary
    print(f"\n{'='*60}")
    print(f"🏥 HEALTH STATUS: {health_report['status']}")
    print(f"{'='*60}")
    
    if health_report['issues']:
        print("⚠️ Issues detected:")
        for issue in health_report['issues']:
            print(f"   - {issue}")
    else:
        print("✅ All systems operating normally")
    
    return health_report


def generate_daily_report(client, ps_id, dev_id, dev_type):
    """
    Generate daily performance report
    
    Args:
        client: SungrowAPI client instance
        ps_id: Power Station ID
        dev_id: Device ID
        dev_type: Device Type
        
    Returns:
        dict: Daily report
    """
    print("📊 Generating daily report...\n")
    
    # Get today's date
    today = datetime.now().strftime('%Y-%m-%d')
    
    # Get historical data for today
    hist_data = client.get_historical_data(
        dev_id=dev_id,
        dev_type=dev_type,
        date_start=today,
        date_end=today,
        date_type=2  # Hourly
    )
    
    # Get PS details
    ps_detail = client.get_ps_detail(ps_id)
    
    report = {
        'date': today,
        'power_station': ps_detail.get('ps_name') if ps_detail else 'N/A',
        'today_energy': ps_detail.get('today_energy') if ps_detail else 'N/A',
        'current_power': ps_detail.get('current_power') if ps_detail else 'N/A',
        'status': 'Generated',
        'timestamp': datetime.now().isoformat()
    }
    
    print(f"📅 Daily Report - {today}")
    print(f"{'='*60}")
    print(f"  Power Station: {report['power_station']}")
    print(f"  Today's Energy: {report['today_energy']} kWh")
    print(f"  Current Power: {report['current_power']} kW")
    print(f"{'='*60}")
    
    return report

print("✅ Monitoring functions created successfully!")

### 13.2 Contoh Penggunaan Fungsi Monitoring

Berikut adalah contoh bagaimana menggunakan fungsi-fungsi monitoring yang telah kita buat.

In [ ]:
# Contoh 1: Check System Health
print("="*70)
print("CONTOH 1: System Health Check")
print("="*70 + "\n")

if all(var in locals() for var in ['client', 'SELECTED_PS_ID']):
    health_status = check_system_health(client, SELECTED_PS_ID)
    print(f"\n📝 Health report saved to variable 'health_status'")
else:
    print("⚠️ Pastikan sudah login dan memiliki PS_ID")

print("\n" + "="*70)
print("CONTOH 2: Generate Daily Report")
print("="*70 + "\n")

# Contoh 2: Generate Daily Report
if all(var in locals() for var in ['client', 'SELECTED_PS_ID', 'SELECTED_DEV_ID', 'SELECTED_DEV_TYPE']):
    daily_report = generate_daily_report(
        client, 
        SELECTED_PS_ID, 
        SELECTED_DEV_ID, 
        SELECTED_DEV_TYPE
    )
    print(f"\n📝 Daily report saved to variable 'daily_report'")
else:
    print("⚠️ Pastikan sudah login dan memiliki device information")

print("\n" + "="*70)
print("CONTOH 3: Real-time Monitoring")
print("="*70 + "\n")

# Contoh 3: Monitor real-time (commented out to avoid long execution)
# Uncomment untuk menjalankan monitoring selama 5 menit dengan interval 30 detik
print("⚠️ Monitoring function ready but not executed (would run for 5 minutes)")
print("   To run monitoring, uncomment the code below:")
print()
print("   # collected_data = monitor_realtime(")
print("   #     client=client,")
print("   #     ps_id=SELECTED_PS_ID,")
print("   #     dev_id=SELECTED_DEV_ID,")
print("   #     dev_type=SELECTED_DEV_TYPE,")
print("   #     interval=30,  # Check every 30 seconds")
print("   #     duration=300  # Run for 5 minutes")
print("   # )")

# Uncomment untuk menjalankan:
# if all(var in locals() for var in ['client', 'SELECTED_PS_ID', 'SELECTED_DEV_ID', 'SELECTED_DEV_TYPE']):
#     collected_data = monitor_realtime(
#         client=client,
#         ps_id=SELECTED_PS_ID,
#         dev_id=SELECTED_DEV_ID,
#         dev_type=SELECTED_DEV_TYPE,
#         interval=30,
#         duration=300
#     )

## ⚠️ Bagian 14: Error Handling dan Troubleshooting

Dalam bekerja dengan API, error adalah hal yang normal. Mari kita pelajari cara menangani error dengan baik.

### 14.1 Common Errors dan Solusinya

In [ ]:
"""
COMMON ERRORS DAN SOLUSINYA
============================

1. LOGIN FAILED (result_code != 1)
   Penyebab:
   - Username atau password salah
   - Region server salah
   - Account locked atau suspended
   
   Solusi:
   - Verify credentials di iSolarCloud website
   - Coba region server yang berbeda
   - Contact Sungrow support jika account issue

2. CONNECTION TIMEOUT
   Penyebab:
   - Network issue
   - Server down atau maintenance
   - Firewall blocking
   
   Solusi:
   - Check internet connection
   - Increase timeout value
   - Try again later
   - Check firewall settings

3. TOKEN EXPIRED
   Penyebab:
   - Token sudah tidak valid (expired)
   - Session timeout
   
   Solusi:
   - Login ulang untuk mendapat token baru
   - Implement auto re-login

4. NO DATA RETURNED (result_data is None)
   Penyebab:
   - Device offline
   - No data available for time period
   - Wrong device ID atau PS ID
   
   Solusi:
   - Verify device is online
   - Check date range
   - Verify IDs are correct

5. RATE LIMITING (Too Many Requests)
   Penyebab:
   - Too many API calls in short time
   
   Solusi:
   - Add delays between requests
   - Implement exponential backoff
   - Reduce polling frequency
"""

# Contoh Error Handling yang Baik
def safe_api_call(func, *args, max_retries=3, retry_delay=2, **kwargs):
    """
    Wrapper function untuk API calls dengan retry logic
    
    Args:
        func: Function to call
        max_retries: Maximum number of retries
        retry_delay: Delay between retries in seconds
        *args, **kwargs: Arguments for the function
        
    Returns:
        Result from function or None if all retries failed
    """
    for attempt in range(max_retries):
        try:
            result = func(*args, **kwargs)
            return result
        except requests.exceptions.Timeout:
            print(f"⏱️ Timeout on attempt {attempt + 1}/{max_retries}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
        except requests.exceptions.ConnectionError:
            print(f"🔌 Connection error on attempt {attempt + 1}/{max_retries}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
        except Exception as e:
            print(f"❌ Error on attempt {attempt + 1}/{max_retries}: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
    
    print("❌ All retry attempts failed")
    return None


# Contoh Class dengan Better Error Handling
class SungrowAPIRobust(SungrowAPI):
    """Extended version dengan better error handling"""
    
    def login_with_retry(self, max_retries=3):
        """Login dengan retry logic"""
        for attempt in range(max_retries):
            try:
                if self.login():
                    return True
                print(f"Login attempt {attempt + 1}/{max_retries} failed")
                time.sleep(2)
            except Exception as e:
                print(f"Login error: {e}")
                time.sleep(2)
        return False
    
    def get_data_safe(self, endpoint, params=None):
        """Get data dengan comprehensive error handling"""
        try:
            # Check if logged in
            if not self.token:
                print("⚠️ Not logged in. Attempting to login...")
                if not self.login_with_retry():
                    return None
            
            # Make request
            result = self._make_request(endpoint, params=params)
            
            # Check if token expired
            if result is None:
                print("⚠️ Request failed. Token might be expired. Re-logging...")
                if self.login_with_retry():
                    result = self._make_request(endpoint, params=params)
            
            return result
            
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            return None

print("✅ Error handling examples and utilities created!")
print("\n📚 Baca docstring di atas untuk memahami common errors dan solusinya.")

## 🚀 Bagian 15: Advanced Use Cases

Mari kita lihat beberapa use case advanced yang bisa Anda implementasikan.

### 15.1 Membuat Dashboard Real-time Sederhana

In [ ]:
def create_dashboard_snapshot(client, ps_id, dev_id, dev_type):
    """
    Create a comprehensive dashboard snapshot
    
    Args:
        client: SungrowAPI instance
        ps_id: Power Station ID
        dev_id: Device ID
        dev_type: Device Type
        
    Returns:
        Dashboard data dictionary
    """
    print("📊 Generating dashboard snapshot...\n")
    
    dashboard = {
        'timestamp': datetime.now().isoformat(),
        'generation_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Get PS details
    ps_detail = client.get_ps_detail(ps_id)
    if ps_detail:
        dashboard['power_station'] = {
            'name': ps_detail.get('ps_name', 'N/A'),
            'capacity': ps_detail.get('ps_capacity', 0),
            'current_power': ps_detail.get('current_power', 0),
            'today_energy': ps_detail.get('today_energy', 0),
            'total_energy': ps_detail.get('total_energy', 0),
            'co2_reduction': ps_detail.get('co2_reduction', 0)
        }
    
    # Get realtime data
    realtime = client.get_realtime_data(dev_id, dev_type)
    if realtime:
        dashboard['realtime_data'] = realtime
    
    # Get alarms
    alarms = client.get_alarm_list(ps_id)
    if alarms:
        active_alarms = [a for a in alarms if a.get('status', '').lower() != 'resolved']
        dashboard['alarms'] = {
            'total': len(alarms),
            'active': len(active_alarms)
        }
    
    # Create visualization
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # Title
    fig.suptitle(f"🌞 Solar Energy Dashboard - {dashboard['generation_time']}", 
                fontsize=18, fontweight='bold')
    
    # Key Metrics Cards
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.axis('off')
    ax1.text(0.5, 0.7, f"{dashboard['power_station']['current_power']}", 
            ha='center', va='center', fontsize=36, fontweight='bold', color='#FF6B6B')
    ax1.text(0.5, 0.3, "Current Power (kW)", 
            ha='center', va='center', fontsize=12, color='gray')
    ax1.add_patch(plt.Rectangle((0.1, 0.1), 0.8, 0.8, fill=False, 
                                edgecolor='#FF6B6B', linewidth=2))
    
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.axis('off')
    ax2.text(0.5, 0.7, f"{dashboard['power_station']['today_energy']}", 
            ha='center', va='center', fontsize=36, fontweight='bold', color='#4ECDC4')
    ax2.text(0.5, 0.3, "Today's Energy (kWh)", 
            ha='center', va='center', fontsize=12, color='gray')
    ax2.add_patch(plt.Rectangle((0.1, 0.1), 0.8, 0.8, fill=False, 
                                edgecolor='#4ECDC4', linewidth=2))
    
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.axis('off')
    alarm_count = dashboard.get('alarms', {}).get('active', 0)
    alarm_color = '#FF6B6B' if alarm_count > 0 else '#95E1D3'
    ax3.text(0.5, 0.7, f"{alarm_count}", 
            ha='center', va='center', fontsize=36, fontweight='bold', color=alarm_color)
    ax3.text(0.5, 0.3, "Active Alarms", 
            ha='center', va='center', fontsize=12, color='gray')
    ax3.add_patch(plt.Rectangle((0.1, 0.1), 0.8, 0.8, fill=False, 
                                edgecolor=alarm_color, linewidth=2))
    
    # System Information
    ax4 = fig.add_subplot(gs[1, :])
    ax4.axis('off')
    info_text = f"""
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    🏭 POWER STATION INFORMATION
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    Name: {dashboard['power_station']['name']}
    Capacity: {dashboard['power_station']['capacity']} kW
    Total Energy Generated: {dashboard['power_station']['total_energy']} kWh
    CO2 Reduction: {dashboard['power_station']['co2_reduction']} tons
    
    Status: {'⚠️ ALERTS ACTIVE' if alarm_count > 0 else '✅ NORMAL OPERATION'}
    """
    ax4.text(0.05, 0.95, info_text, 
            fontsize=11, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))
    
    # Capacity Gauge
    ax5 = fig.add_subplot(gs[2, 0])
    capacity = float(dashboard['power_station']['capacity']) if dashboard['power_station']['capacity'] else 1
    current = float(dashboard['power_station']['current_power']) if dashboard['power_station']['current_power'] else 0
    percentage = (current / capacity * 100) if capacity > 0 else 0
    
    colors = ['#FF6B6B' if percentage < 30 else '#FFD93D' if percentage < 70 else '#95E1D3']
    ax5.pie([percentage, 100-percentage], colors=[colors[0], '#E8E8E8'], 
           startangle=90, counterclock=False)
    ax5.text(0, 0, f'{percentage:.1f}%', ha='center', va='center', 
            fontsize=20, fontweight='bold')
    ax5.set_title('Capacity Utilization', fontsize=12, fontweight='bold')
    
    # Performance Indicator
    ax6 = fig.add_subplot(gs[2, 1:])
    categories = ['Current\nPower', 'Today\nEnergy', 'Total\nEnergy', 'CO2\nReduction']
    values = [
        float(dashboard['power_station']['current_power']) if dashboard['power_station']['current_power'] else 0,
        float(dashboard['power_station']['today_energy']) if dashboard['power_station']['today_energy'] else 0,
        float(dashboard['power_station']['total_energy']) / 1000 if dashboard['power_station']['total_energy'] else 0,  # Convert to MWh
        float(dashboard['power_station']['co2_reduction']) if dashboard['power_station']['co2_reduction'] else 0
    ]
    
    bars = ax6.bar(categories, values, color=['#FF6B6B', '#4ECDC4', '#95E1D3', '#F38181'], 
                  alpha=0.7, edgecolor='black')
    ax6.set_title('Performance Metrics', fontsize=12, fontweight='bold')
    ax6.set_ylabel('Value')
    ax6.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.show()
    
    print("✅ Dashboard created successfully!")
    return dashboard


# Example usage (if data is available)
if all(var in locals() for var in ['client', 'SELECTED_PS_ID', 'SELECTED_DEV_ID', 'SELECTED_DEV_TYPE']):
    try:
        dashboard_data = create_dashboard_snapshot(
            client, 
            SELECTED_PS_ID, 
            SELECTED_DEV_ID, 
            SELECTED_DEV_TYPE
        )
        print("\n📝 Dashboard data tersimpan di variable 'dashboard_data'")
    except Exception as e:
        print(f"⚠️ Dashboard generation failed: {e}")
        print("   This might be due to missing or invalid data from API")
else:
    print("⚠️ Dashboard function ready but not executed")
    print("   Ensure you have valid client, PS_ID, and device information")

### 15.2 Analisis Performa dan Prediksi

In [ ]:
def analyze_performance(client, ps_id, dev_id, dev_type, days=30):
    """
    Analyze system performance over a period
    
    Args:
        client: SungrowAPI instance
        ps_id: Power Station ID
        dev_id: Device ID
        dev_type: Device Type
        days: Number of days to analyze
        
    Returns:
        Performance analysis dictionary
    """
    print(f"📊 Analyzing performance for last {days} days...\n")
    
    # Get historical data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    hist_data = client.get_historical_data(
        dev_id=dev_id,
        dev_type=dev_type,
        date_start=start_date.strftime('%Y-%m-%d'),
        date_end=end_date.strftime('%Y-%m-%d'),
        date_type=3  # Daily data
    )
    
    if not hist_data or 'list' not in hist_data:
        print("❌ No historical data available")
        return None
    
    # Convert to DataFrame
    df = pd.DataFrame(hist_data['list'])
    
    # Analysis
    analysis = {
        'period': f"{start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}",
        'total_days': days,
        'data_points': len(df)
    }
    
    # Find energy column (name may vary)
    energy_cols = [col for col in df.columns if 'energy' in col.lower() or 'kwh' in col.lower()]
    if energy_cols:
        energy_col = energy_cols[0]
        df[energy_col] = pd.to_numeric(df[energy_col], errors='coerce')
        
        analysis['total_energy'] = df[energy_col].sum()
        analysis['avg_daily_energy'] = df[energy_col].mean()
        analysis['max_daily_energy'] = df[energy_col].max()
        analysis['min_daily_energy'] = df[energy_col].min()
        analysis['std_dev'] = df[energy_col].std()
        
        # Performance consistency (coefficient of variation)
        cv = (analysis['std_dev'] / analysis['avg_daily_energy']) * 100 if analysis['avg_daily_energy'] > 0 else 0
        analysis['consistency'] = 'High' if cv < 20 else 'Medium' if cv < 40 else 'Low'
        analysis['cv_percentage'] = cv
        
        # Trend analysis (simple linear regression)
        df['day_index'] = range(len(df))
        if len(df) > 2:
            z = np.polyfit(df['day_index'].values, df[energy_col].values, 1)
            analysis['trend_slope'] = z[0]
            analysis['trend'] = 'Improving' if z[0] > 0 else 'Declining' if z[0] < 0 else 'Stable'
        
        # Create visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle(f'📈 Performance Analysis - Last {days} Days', 
                    fontsize=16, fontweight='bold')
        
        # Plot 1: Daily Energy Production
        axes[0, 0].plot(df.index, df[energy_col], marker='o', linewidth=2, color='#4ECDC4')
        axes[0, 0].set_title('Daily Energy Production', fontsize=12, fontweight='bold')
        axes[0, 0].set_xlabel('Day')
        axes[0, 0].set_ylabel('Energy (kWh)')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Add trend line if available
        if 'trend_slope' in analysis:
            trend_line = np.poly1d(z)
            axes[0, 0].plot(df['day_index'], trend_line(df['day_index']), 
                          "r--", linewidth=2, label=f"Trend: {analysis['trend']}")
            axes[0, 0].legend()
        
        # Plot 2: Distribution
        axes[0, 1].hist(df[energy_col].dropna(), bins=20, color='#95E1D3', 
                       edgecolor='black', alpha=0.7)
        axes[0, 1].axvline(analysis['avg_daily_energy'], color='red', 
                          linestyle='--', linewidth=2, label='Mean')
        axes[0, 1].set_title('Energy Distribution', fontsize=12, fontweight='bold')
        axes[0, 1].set_xlabel('Energy (kWh)')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3, axis='y')
        
        # Plot 3: Box Plot
        axes[1, 0].boxplot(df[energy_col].dropna(), vert=True)
        axes[1, 0].set_title('Energy Production Box Plot', fontsize=12, fontweight='bold')
        axes[1, 0].set_ylabel('Energy (kWh)')
        axes[1, 0].grid(True, alpha=0.3, axis='y')
        
        # Plot 4: Statistics Summary
        axes[1, 1].axis('off')
        stats_text = f"""
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        📊 PERFORMANCE STATISTICS
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        
        Period: {analysis['period']}
        Data Points: {analysis['data_points']} days
        
        📈 Energy Production:
           Total: {analysis['total_energy']:.2f} kWh
           Average/Day: {analysis['avg_daily_energy']:.2f} kWh
           Maximum: {analysis['max_daily_energy']:.2f} kWh
           Minimum: {analysis['min_daily_energy']:.2f} kWh
           Std Dev: {analysis['std_dev']:.2f} kWh
        
        🎯 Performance:
           Consistency: {analysis['consistency']}
           Variability: {analysis['cv_percentage']:.1f}%
           Trend: {analysis['trend']}
        
        💡 Estimated Monthly: {analysis['avg_daily_energy'] * 30:.2f} kWh
        💡 Estimated Yearly: {analysis['avg_daily_energy'] * 365:.2f} kWh
        """
        axes[1, 1].text(0.05, 0.95, stats_text, 
                       fontsize=10, verticalalignment='top', fontfamily='monospace',
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
        
        plt.tight_layout()
        plt.show()
    
    print("\n✅ Performance analysis completed!")
    return analysis


# Example usage
if all(var in locals() for var in ['client', 'SELECTED_PS_ID', 'SELECTED_DEV_ID', 'SELECTED_DEV_TYPE']):
    try:
        # Analyze last 30 days
        performance_analysis = analyze_performance(
            client, 
            SELECTED_PS_ID, 
            SELECTED_DEV_ID, 
            SELECTED_DEV_TYPE,
            days=30
        )
        
        if performance_analysis:
            print("\n📝 Analysis saved to 'performance_analysis' variable")
    except Exception as e:
        print(f"⚠️ Analysis failed: {e}")
        print("   This might be due to insufficient historical data")
else:
    print("⚠️ Performance analysis function ready")
    print("   Run previous cells to get required data")

### 15.3 Alert System dan Notifikasi

In [ ]:
class AlertSystem:
    """
    Simple alert system untuk monitoring Sungrow devices
    
    Ini adalah contoh sederhana. Untuk production, Anda bisa integrate dengan:
    - Email (SMTP)
    - SMS gateway
    - Telegram bot
    - Slack webhook
    - WhatsApp Business API
    """
    
    def __init__(self, client, ps_id):
        self.client = client
        self.ps_id = ps_id
        self.alert_log = []
    
    def check_alerts(self):
        """Check untuk kondisi yang memerlukan alert"""
        alerts = []
        timestamp = datetime.now()
        
        # Check alarms
        alarms = self.client.get_alarm_list(self.ps_id)
        if alarms:
            active_alarms = [a for a in alarms if a.get('status', '').lower() != 'resolved']
            if active_alarms:
                alert = {
                    'timestamp': timestamp,
                    'type': 'ALARM',
                    'severity': 'HIGH',
                    'message': f"{len(active_alarms)} active alarm(s) detected",
                    'details': active_alarms[:5]  # First 5 alarms
                }
                alerts.append(alert)
        
        # Check PS details
        ps_detail = self.client.get_ps_detail(self.ps_id)
        if ps_detail:
            current_power = float(ps_detail.get('current_power', 0))
            capacity = float(ps_detail.get('ps_capacity', 1))
            
            # Check if power is too low during day
            hour = timestamp.hour
            if 10 <= hour <= 15:  # Peak hours
                if current_power < capacity * 0.1:  # Less than 10% capacity
                    alert = {
                        'timestamp': timestamp,
                        'type': 'LOW_PERFORMANCE',
                        'severity': 'MEDIUM',
                        'message': f"Low power output: {current_power} kW ({(current_power/capacity)*100:.1f}% of capacity)",
                        'details': ps_detail
                    }
                    alerts.append(alert)
        
        # Check devices
        devices = self.client.get_device_list(self.ps_id)
        if devices:
            offline_devices = [d for d in devices if d.get('dev_status', '').lower() == 'offline']
            if offline_devices:
                alert = {
                    'timestamp': timestamp,
                    'type': 'DEVICE_OFFLINE',
                    'severity': 'CRITICAL',
                    'message': f"{len(offline_devices)} device(s) offline",
                    'details': offline_devices
                }
                alerts.append(alert)
        
        # Save to log
        for alert in alerts:
            self.alert_log.append(alert)
        
        return alerts
    
    def print_alerts(self, alerts):
        """Print alerts dengan format yang bagus"""
        if not alerts:
            print("✅ No alerts - System operating normally")
            return
        
        print(f"\n{'='*70}")
        print(f"🚨 ALERT REPORT - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*70}\n")
        
        for idx, alert in enumerate(alerts, 1):
            severity_icon = {
                'LOW': '💚',
                'MEDIUM': '💛',
                'HIGH': '🧡',
                'CRITICAL': '🔴'
            }
            
            icon = severity_icon.get(alert['severity'], '⚠️')
            
            print(f"{icon} Alert #{idx}")
            print(f"{'─'*70}")
            print(f"  Type: {alert['type']}")
            print(f"  Severity: {alert['severity']}")
            print(f"  Message: {alert['message']}")
            print(f"  Time: {alert['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")
            print()
        
        print(f"{'='*70}\n")
    
    def send_notification(self, alert):
        """
        Send notification (placeholder - implement actual notification here)
        
        Untuk implementasi nyata, Anda bisa:
        1. Email via SMTP
        2. SMS via Twilio
        3. Telegram via Bot API
        4. Slack via Webhook
        """
        print(f"📧 [NOTIFICATION] {alert['type']}: {alert['message']}")
        # TODO: Implement actual notification sending
    
    def get_alert_summary(self):
        """Get summary of all alerts"""
        if not self.alert_log:
            return "No alerts in log"
        
        df = pd.DataFrame(self.alert_log)
        summary = {
            'total_alerts': len(self.alert_log),
            'by_type': df['type'].value_counts().to_dict(),
            'by_severity': df['severity'].value_counts().to_dict(),
            'latest': self.alert_log[-1]
        }
        return summary


# Example usage
print("🔔 Alert System Class created successfully!")
print("\n📝 Example usage:")
print("""
# Initialize alert system
alert_system = AlertSystem(client, SELECTED_PS_ID)

# Check for alerts
alerts = alert_system.check_alerts()

# Print alerts
alert_system.print_alerts(alerts)

# Get summary
summary = alert_system.get_alert_summary()
print(summary)
""")

## 📚 Bagian 16: Best Practices dan Tips

Berikut adalah best practices yang sebaiknya Anda ikuti saat bekerja dengan Sungrow API.

### 16.1 Security Best Practices

```python
# ============================================
# BEST PRACTICES - SECURITY
# ============================================

# ✅ DO: Use environment variables
import os
USERNAME = os.getenv('SUNGROW_USERNAME')
PASSWORD = os.getenv('SUNGROW_PASSWORD')

# ✅ DO: Use config files (add to .gitignore)
import json
with open('config.json') as f:
    config = json.load(f)
    USERNAME = config['username']
    PASSWORD = config['password']

# ❌ DON'T: Hardcode credentials
USERNAME = "my_email@example.com"  # BAD!
PASSWORD = "my_password123"        # BAD!

# ✅ DO: Implement token refresh
def ensure_logged_in(client):
    if not client.token:
        client.login()
    return True

# ✅ DO: Use HTTPS only
# Sungrow API sudah menggunakan HTTPS by default

# ============================================
# BEST PRACTICES - PERFORMANCE
# ============================================

# ✅ DO: Cache data when appropriate
from functools import lru_cache
from datetime import datetime, timedelta

@lru_cache(maxsize=128)
def get_cached_ps_list(client_id, cache_time):
    # Cache will invalidate when cache_time changes
    return client.get_power_stations()

# Use it:
cache_time = datetime.now().replace(minute=0, second=0, microsecond=0)
ps_list = get_cached_ps_list(id(client), cache_time)

# ✅ DO: Implement rate limiting
import time

def rate_limited_call(func, min_interval=1.0):
    last_call = [0]
    
    def wrapper(*args, **kwargs):
        elapsed = time.time() - last_call[0]
        if elapsed < min_interval:
            time.sleep(min_interval - elapsed)
        result = func(*args, **kwargs)
        last_call[0] = time.time()
        return result
    
    return wrapper

# ✅ DO: Batch requests when possible
def get_all_device_data(client, ps_list):
    all_data = {}
    for ps in ps_list:
        devices = client.get_device_list(ps['ps_id'])
        all_data[ps['ps_id']] = devices
        time.sleep(0.5)  # Rate limiting
    return all_data

# ❌ DON'T: Make too many rapid requests
# for i in range(1000):
#     client.get_realtime_data(...)  # BAD! Will hit rate limits

# ============================================
# BEST PRACTICES - ERROR HANDLING
# ============================================

# ✅ DO: Always handle exceptions
try:
    data = client.get_realtime_data(dev_id, dev_type)
    if data:
        # Process data
        pass
    else:
        print("No data returned")
except Exception as e:
    print(f"Error: {e}")
    # Log error, retry, or alert

# ✅ DO: Validate data before using
def safe_get_value(data, key, default=0, value_type=float):
    try:
        value = data.get(key, default)
        return value_type(value)
    except (ValueError, TypeError):
        return default

# ✅ DO: Implement retry logic
def retry_request(func, max_retries=3, delay=2):
    for attempt in range(max_retries):
        try:
            return func()
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            time.sleep(delay * (attempt + 1))

# ============================================
# BEST PRACTICES - DATA MANAGEMENT
# ============================================

# ✅ DO: Use proper data types
df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['power'] = pd.to_numeric(df['power'], errors='coerce')

# ✅ DO: Handle missing data
df = df.fillna(0)  # or df.dropna()

# ✅ DO: Regular backups
def backup_data(df, backup_folder='backups'):
    os.makedirs(backup_folder, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"{backup_folder}/data_backup_{timestamp}.csv"
    df.to_csv(filename, index=False)
    return filename

# ✅ DO: Clean up old data
def cleanup_old_files(folder, days=30):
    cutoff = datetime.now() - timedelta(days=days)
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if os.path.isfile(filepath):
            mtime = datetime.fromtimestamp(os.path.getmtime(filepath))
            if mtime < cutoff:
                os.remove(filepath)

# ============================================
# BEST PRACTICES - LOGGING
# ============================================

# ✅ DO: Implement logging
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('sungrow_api.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger('SungrowAPI')

# Use it:
logger.info("Starting data collection")
logger.warning("Low power output detected")
logger.error("API request failed")

# ============================================
# BEST PRACTICES - MONITORING
# ============================================

# ✅ DO: Track API usage
api_calls = {
    'total': 0,
    'success': 0,
    'failed': 0,
    'last_call': None
}

def tracked_api_call(func):
    def wrapper(*args, **kwargs):
        api_calls['total'] += 1
        api_calls['last_call'] = datetime.now()
        try:
            result = func(*args, **kwargs)
            api_calls['success'] += 1
            return result
        except Exception as e:
            api_calls['failed'] += 1
            raise
    return wrapper

# ✅ DO: Monitor system health
def health_check(client, ps_id):
    checks = {
        'api_connection': False,
        'devices_online': False,
        'no_critical_alarms': False
    }
    
    try:
        # Check API connection
        ps_list = client.get_power_stations()
        checks['api_connection'] = ps_list is not None
        
        # Check devices
        devices = client.get_device_list(ps_id)
        if devices:
            online = sum(1 for d in devices if d.get('dev_status') != 'offline')
            checks['devices_online'] = online == len(devices)
        
        # Check alarms
        alarms = client.get_alarm_list(ps_id)
        if alarms:
            critical = sum(1 for a in alarms if a.get('alarm_level') == 'critical')
            checks['no_critical_alarms'] = critical == 0
        
        return all(checks.values()), checks
    except Exception as e:
        logger.error(f"Health check failed: {e}")
        return False, checks
```

### Tips Tambahan:

1. **Testing**: Selalu test code Anda dengan small dataset dulu
2. **Documentation**: Document semua custom functions dan classes
3. **Version Control**: Use Git untuk track changes
4. **Monitoring**: Setup monitoring untuk production systems
5. **Backup**: Regular backup data penting
6. **Updates**: Keep library dependencies updated
7. **Error Logs**: Maintain comprehensive error logs
8. **Performance**: Profile code untuk identify bottlenecks

## 🎯 Bagian 17: Contoh Project Lengkap

Mari kita gabungkan semua yang telah dipelajari menjadi sebuah aplikasi monitoring lengkap.

In [ ]:
class SungrowMonitoringSystem:
    """
    Comprehensive Solar Monitoring System
    
    Features:
    - Automated data collection
    - Real-time monitoring
    - Alert system
    - Data export
    - Performance analysis
    - Dashboard generation
    """
    
    def __init__(self, username, password, region='global'):
        self.client = SungrowAPI(username, password, region)
        self.ps_id = None
        self.devices = []
        self.data_history = []
        
        # Configuration
        self.config = {
            'check_interval': 300,  # 5 minutes
            'export_folder': 'monitoring_data',
            'alert_threshold': {
                'low_power_pct': 10,  # % of capacity
                'high_temp': 60  # Celsius
            }
        }
        
        # Create export folder
        os.makedirs(self.config['export_folder'], exist_ok=True)
    
    def initialize(self):
        """Initialize the monitoring system"""
        print("🚀 Initializing Sungrow Monitoring System...")
        
        # Login
        if not self.client.login():
            print("❌ Login failed!")
            return False
        
        # Get power stations
        ps_list = self.client.get_power_stations()
        if not ps_list:
            print("❌ No power stations found!")
            return False
        
        # Use first power station
        self.ps_id = ps_list[0]['ps_id']
        print(f"✅ Using Power Station: {ps_list[0]['ps_name']}")
        
        # Get devices
        self.devices = self.client.get_device_list(self.ps_id)
        if self.devices:
            print(f"✅ Found {len(self.devices)} device(s)")
        
        print("✅ Initialization complete!\n")
        return True
    
    def collect_data(self):
        """Collect current data from all devices"""
        timestamp = datetime.now()
        collection = {
            'timestamp': timestamp.isoformat(),
            'ps_detail': None,
            'devices_data': [],
            'alarms': None
        }
        
        # Get PS details
        collection['ps_detail'] = self.client.get_ps_detail(self.ps_id)
        
        # Get device data
        for device in self.devices:
            dev_data = self.client.get_realtime_data(
                device['dev_id'],
                device.get('dev_type', 1)
            )
            if dev_data:
                collection['devices_data'].append({
                    'device_id': device['dev_id'],
                    'device_name': device.get('dev_name'),
                    'data': dev_data
                })
        
        # Get alarms
        collection['alarms'] = self.client.get_alarm_list(self.ps_id)
        
        # Save to history
        self.data_history.append(collection)
        
        return collection
    
    def check_alerts(self, data):
        """Check for alert conditions"""
        alerts = []
        
        # Check alarms
        if data['alarms']:
            active = [a for a in data['alarms'] if a.get('status') != 'resolved']
            if active:
                alerts.append({
                    'type': 'ALARM',
                    'severity': 'HIGH',
                    'message': f"{len(active)} active alarm(s)"
                })
        
        # Check power output
        if data['ps_detail']:
            current_power = float(data['ps_detail'].get('current_power', 0))
            capacity = float(data['ps_detail'].get('ps_capacity', 1))
            
            hour = datetime.now().hour
            if 9 <= hour <= 16:  # Daytime
                if current_power < capacity * (self.config['alert_threshold']['low_power_pct'] / 100):
                    alerts.append({
                        'type': 'LOW_PERFORMANCE',
                        'severity': 'MEDIUM',
                        'message': f"Low power: {current_power:.1f} kW"
                    })
        
        return alerts
    
    def export_data(self, data, filename_prefix='monitoring'):
        """Export collected data"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Export to JSON
        json_file = f"{self.config['export_folder']}/{filename_prefix}_{timestamp}.json"
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False, default=str)
        
        return json_file
    
    def generate_report(self):
        """Generate monitoring report"""
        if not self.data_history:
            print("⚠️ No data collected yet")
            return None
        
        report = {
            'generated_at': datetime.now().isoformat(),
            'monitoring_period': {
                'start': self.data_history[0]['timestamp'],
                'end': self.data_history[-1]['timestamp']
            },
            'data_points': len(self.data_history),
            'summary': {}
        }
        
        # Calculate averages
        if self.data_history[-1]['ps_detail']:
            latest = self.data_history[-1]['ps_detail']
            report['summary'] = {
                'current_power': latest.get('current_power'),
                'today_energy': latest.get('today_energy'),
                'total_energy': latest.get('total_energy')
            }
        
        return report
    
    def run_monitoring_cycle(self):
        """Run one monitoring cycle"""
        print(f"\n{'='*70}")
        print(f"🔄 Monitoring Cycle - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*70}\n")
        
        # Collect data
        data = self.collect_data()
        print("✅ Data collected")
        
        # Check alerts
        alerts = self.check_alerts(data)
        if alerts:
            print(f"\n⚠️ {len(alerts)} alert(s) detected:")
            for alert in alerts:
                print(f"   [{alert['severity']}] {alert['type']}: {alert['message']}")
        else:
            print("✅ No alerts - System normal")
        
        # Show key metrics
        if data['ps_detail']:
            ps = data['ps_detail']
            print(f"\n📊 Current Status:")
            print(f"   Power: {ps.get('current_power', 'N/A')} kW")
            print(f"   Today Energy: {ps.get('today_energy', 'N/A')} kWh")
            print(f"   Devices: {len(self.devices)} online")
        
        return data, alerts
    
    def start_monitoring(self, duration_minutes=60, check_interval_seconds=None):
        """
        Start continuous monitoring
        
        Args:
            duration_minutes: How long to monitor (minutes)
            check_interval_seconds: Interval between checks (seconds)
        """
        if check_interval_seconds:
            self.config['check_interval'] = check_interval_seconds
        
        print(f"\n{'='*70}")
        print(f"🚀 Starting Monitoring System")
        print(f"{'='*70}")
        print(f"Duration: {duration_minutes} minutes")
        print(f"Check interval: {self.config['check_interval']} seconds")
        print(f"Press Ctrl+C to stop\n")
        
        start_time = time.time()
        end_time = start_time + (duration_minutes * 60)
        
        try:
            while time.time() < end_time:
                data, alerts = self.run_monitoring_cycle()
                
                # Export if needed
                if alerts:
                    self.export_data(data, 'alert_data')
                
                # Wait for next cycle
                time.sleep(self.config['check_interval'])
        
        except KeyboardInterrupt:
            print("\n\n⏹️ Monitoring stopped by user")
        
        # Generate final report
        report = self.generate_report()
        if report:
            report_file = self.export_data(report, 'final_report')
            print(f"\n📄 Report saved: {report_file}")
        
        print(f"\n✅ Monitoring session complete!")
        print(f"   Total data points: {len(self.data_history)}")
        print(f"   Duration: {(time.time() - start_time) / 60:.1f} minutes")


print("✅ SungrowMonitoringSystem class created!")
print("\n📝 Usage example:")
print("""
# Create and initialize system
monitor = SungrowMonitoringSystem(USERNAME, PASSWORD, REGION)
if monitor.initialize():
    # Run single cycle
    data, alerts = monitor.run_monitoring_cycle()
    
    # Or run continuous monitoring
    # monitor.start_monitoring(duration_minutes=30, check_interval_seconds=60)
""")

## 📖 Bagian 18: Resources dan Referensi

### Dokumentasi dan Links

1. **Sungrow Official**
   - Website: https://www.sungrowpower.com/
   - iSolarCloud: https://www.isolarcloud.com/

2. **API Documentation**
   - Sungrow tidak menyediakan public API documentation
   - API ini adalah reverse-engineered dari web interface
   - Endpoint dan parameter bisa berubah tanpa pemberitahuan

3. **Python Libraries**
   - requests: https://docs.python-requests.org/
   - pandas: https://pandas.pydata.org/docs/
   - matplotlib: https://matplotlib.org/stable/contents.html
   - seaborn: https://seaborn.pydata.org/

4. **Community Resources**
   - GitHub: Cari "sungrow api python" untuk community projects
   - Stack Overflow: Tag sungrow
   - Home Assistant Integration: https://www.home-assistant.io/

### Topik Lanjutan untuk Dipelajari

1. **Web Development**
   - Flask/Django untuk web dashboard
   - Streamlit untuk quick prototyping
   - FastAPI untuk REST API

2. **Database Integration**
   - PostgreSQL/MySQL untuk data storage
   - InfluxDB untuk time-series data
   - MongoDB untuk JSON documents

3. **Visualization**
   - Grafana untuk monitoring dashboards
   - Plotly Dash untuk interactive dashboards
   - Power BI / Tableau untuk business intelligence

4. **Automation**
   - Cron jobs (Linux) atau Task Scheduler (Windows)
   - Apache Airflow untuk complex workflows
   - Docker untuk containerization

5. **Cloud Deployment**
   - AWS (Lambda, EC2, RDS)
   - Google Cloud Platform
   - Microsoft Azure
   - Heroku untuk simple deployments

6. **Advanced Analytics**
   - Machine Learning untuk predictive maintenance
   - Anomaly detection
   - Weather data integration
   - Energy forecasting

### Troubleshooting Common Issues

**Issue 1: Login gagal terus**
- Verify credentials di website iSolarCloud
- Coba region server yang berbeda
- Check apakah account di-lock

**Issue 2: Data tidak lengkap**
- Beberapa device mungkin tidak support semua parameter
- Check device model dan capabilities
- Verify device online status

**Issue 3: Connection timeout**
- Check internet connection
- Increase timeout value dalam requests
- Try again during off-peak hours

**Issue 4: Rate limiting**
- Reduce polling frequency
- Implement exponential backoff
- Batch requests when possible

**Issue 5: Token expired**
- Implement auto re-login
- Token biasanya valid 24 jam
- Store token expiry time

### Contact & Support

Untuk support dari Sungrow:
- Customer Service: Check regional website
- Technical Support: Hubungi installer Anda
- Emergency: Check device manual

Untuk issues dengan code ini:
- Review error messages carefully
- Check Python version compatibility
- Verify all dependencies installed
- Enable logging untuk detailed debugging

## 🎓 Kesimpulan

Selamat! Anda telah menyelesaikan tutorial lengkap Sungrow API! 🎉

### Apa yang Telah Anda Pelajari:

✅ **Dasar-dasar**
- Setup dan instalasi library Python
- Authentication dengan Sungrow API
- Struktur data dan endpoints

✅ **Data Collection**
- Mengambil daftar power station dan devices
- Real-time data monitoring
- Historical data retrieval
- Alarm dan event logs

✅ **Data Analysis**
- Data manipulation dengan pandas
- Statistical analysis
- Performance metrics calculation
- Trend analysis

✅ **Visualization**
- Chart creation dengan matplotlib
- Dashboard design
- Data comparison dan reporting

✅ **Advanced Features**
- Error handling dan retry logic
- Automated monitoring system
- Alert system
- Data export dan backup
- Best practices dan security

### Next Steps:

1. **Practice**: Implementasikan code dengan data Anda sendiri
2. **Customize**: Sesuaikan functions untuk kebutuhan spesifik Anda
3. **Expand**: Tambahkan features seperti database integration, web interface
4. **Deploy**: Deploy ke production environment
5. **Monitor**: Setup continuous monitoring system

### Tips untuk Sukses:

💡 **Start Small**: Mulai dengan simple script, gradually add complexity
💡 **Test Often**: Test setiap function sebelum integrate
💡 **Document**: Document code dan decisions Anda
💡 **Version Control**: Use Git untuk track changes
💡 **Backup**: Always backup data penting
💡 **Security**: Never expose credentials
💡 **Monitor**: Keep eye on system health

### Remember:

- API ini **unofficial** dan bisa berubah
- **Rate limiting** exists - don't abuse the API
- **Error handling** is critical untuk production
- **Security** harus prioritas utama
- **Testing** saves time in the long run

---

### 🙏 Terima Kasih!

Terima kasih telah mengikuti tutorial ini. Semoga bermanfaat untuk project Anda!

**Good luck dengan monitoring solar energy system Anda!** ☀️🔋⚡

---

### 📝 Quick Reference Card

```python
# Quick Start Template
from datetime import datetime
import requests
import pandas as pd

# 1. Initialize
client = SungrowAPI(username, password, region)
client.login()

# 2. Get Power Stations
ps_list = client.get_power_stations()
ps_id = ps_list[0]['ps_id']

# 3. Get Devices
devices = client.get_device_list(ps_id)
dev_id = devices[0]['dev_id']
dev_type = devices[0]['dev_type']

# 4. Get Real-time Data
realtime = client.get_realtime_data(dev_id, dev_type)

# 5. Get Historical Data
historical = client.get_historical_data(
    dev_id, dev_type,
    date_start='2024-01-01',
    date_end='2024-01-31',
    date_type=2  # Hourly
)

# 6. Get Alarms
alarms = client.get_alarm_list(ps_id)

# 7. Export to CSV
df = pd.DataFrame(historical['list'])
df.to_csv('data.csv', index=False)
```

---

**© 2026 - Tutorial Sungrow API untuk PLN Internship**